# Airbnb Berlin — Price Prediction (EDA)

**Goal.** Understand which factors explain nightly price in Berlin listings and prepare a clean feature table for a simple baseline model.

**This notebook delivers**
- Load InsideAirbnb listings (Berlin)
- Clean messy fields (price strings, bathrooms)
- Quick exploratory plots (distribution, neighbourhoods, room type)
- Export a compact, cleaned dataset for model training

In [ ]:
# Libraries & display options
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 120)

print("Pandas:", pd.__version__)

## Data snapshot

**Source.** InsideAirbnb `data/listings.csv.gz` (raw scrape).  
For EDA plots I restrict prices to **10–500 €** to avoid distortion from rare extremes.

In [ ]:
# Load the raw InsideAirbnb file (run from notebooks/ directory)
PATH = "../data/listings.csv.gz"
df = pd.read_csv(PATH, low_memory=False)

print("Rows, Cols:", df.shape)
df.head()

### What’s in the data?

I scan column names and dtypes to spot useful features and cleaning needs. Notable fields:
- **price** — text with currency symbols / thousand separators
- **neighbourhood / neighbourhood_cleansed** — location (both can exist)
- **room_type** — category (Entire home/apt, Private room, …)
- Counts: `accommodates`, `bedrooms`, `number_of_reviews`, `availability_365`, …

In [ ]:
# Quick schema overview
df.info()                     # shows dtypes + non-nulls
df.columns[:40].tolist()      # first 40 column names

## Price cleaning (turn strings like “€1,234” into floats)

Raw prices arrive as strings. I strip currency symbols and separators and parse to **euros**.  
Result check: `price` becomes numeric; the summary typically shows a **median ≈ 107 €**.

In [ ]:
def to_euro(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace("\xa0","").replace("€","").replace("$","").strip()
    if "," in s and "." in s:            # z.B. 1.234,56  -> 1234.56
        s = s.replace(".", "").replace(",", ".")
    elif "," in s:                       # z.B. 1,234     -> 1234
        s = s.replace(",", "")
    elif "." in s and len(s.split(".")[-1]) == 3:  # z.B. 1.234 -> 1234
        s = s.replace(".", "")
    try:
        return float(s)
    except ValueError:
        return np.nan
# apply price cleaning
if "price" not in df.columns:
    raise KeyError("Column 'price' not found.")
df["price"] = df["price"].apply(to_euro)
df["price"].describe()


## Normalise bathrooms & choose a stable neighbourhood column

- **Bathrooms.** Listings may have numeric `bathrooms` or textual `bathrooms_text` (e.g. “1 bath”). I create `bathrooms_num` robustly for both cases.
- **Neighbourhood.** Prefer `neighbourhood_cleansed` if present; otherwise fall back to `neighbourhood` (robust across InsideAirbnb snapshots).

In [ ]:
# 1) Pick neighbourhood column available in this snapshot
if "neighbourhood_cleansed" in df.columns:
    NBH_COL = "neighbourhood_cleansed"
elif "neighbourhood" in df.columns:
    NBH_COL = "neighbourhood"
else:
    NBH_COL = None
print("Neighbourhood column:", NBH_COL)

# 2) Standardize name to 'neighbourhood' (easier downstream)
if NBH_COL and NBH_COL != "neighbourhood":
    df["neighbourhood"] = df[NBH_COL]
    NBH_COL = "neighbourhood"

# 3) Simple & robust bathrooms parser (handles "1.5 baths", "half bath", "1,5")
def simple_bath(x):
    """Return numeric bathrooms (e.g., '1.5 baths' -> 1.5)."""
    if pd.isna(x):
        return np.nan
    s = str(x).lower().replace(",", ".")  # 1,5 -> 1.5
    if "half" in s and not any(ch.isdigit() for ch in s):
        return 0.5
    buf = []
    for ch in s:
        if ch.isdigit() or ch == ".":
            buf.append(ch)
        elif buf:
            break
    try:
        return float("".join(buf)) if buf else np.nan
    except ValueError:
        return np.nan

# 4) Create bathrooms_num from whichever column exists
if "bathrooms" in df.columns:
    df["bathrooms_num"] = pd.to_numeric(df["bathrooms"], errors="coerce")
elif "bathrooms_text" in df.columns:
    df["bathrooms_num"] = df["bathrooms_text"].apply(simple_bath)
else:
    df["bathrooms_num"] = np.nan

# 5) Make sure typical numeric columns are numeric (simple & safe)
for c in ["accommodates","bedrooms","minimum_nights",
          "number_of_reviews","reviews_per_month","availability_365"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 6) Cast common categoricals (optional, saves memory)
if "room_type" in df.columns:
    df["room_type"] = df["room_type"].astype("category")
if NBH_COL:
    df[NBH_COL] = df[NBH_COL].astype("category")

## EDA subset (for plots only)

For clearer visuals, plots use listings with **10–500 €**.  
(The exported training slice uses the cleaned values; the 10–500 € window is only for charts.)

In [ ]:
eda = df.loc[df["price"].between(10, 500)].copy()
print("EDA shape:", eda.shape, "| missing price %:", f"{eda['price'].isna().mean()*100:.1f}%")
eda["price"].describe()

### What do prices look like?

The distribution is **right-skewed**: most listings ~60–180 €, with a long tail to higher prices.  
**Mean ≈ 129 €**, **Median ≈ 107 €** → the median represents the “typical” price better.

In [ ]:
prices = eda["price"].dropna()

plt.figure(figsize=(8,4))
plt.hist(prices, bins=60)
mean, median = prices.mean(), prices.median()
plt.axvline(mean,   linestyle="--", linewidth=1, label=f"Mean ≈ {mean:.0f}€")
plt.axvline(median, linestyle="-.", linewidth=1, label=f"Median ≈ {median:.0f}€")
plt.legend(loc="upper right")
plt.title("Airbnb Price Distribution – Berlin")
plt.xlabel("Price (€)"); 
plt.ylabel("Count"); 
plt.xlim(0, 500)
plt.tight_layout(); 
plt.show()

## Which neighbourhoods are more expensive?

I compute **average price per neighbourhood**, keeping places with at least **30 listings** for stable means.  
In Berlin, central areas (e.g. *Regierungsviertel*, parts of *Prenzlauer Berg*) trend higher; peripheral areas lower.

In [ ]:
MIN_LISTINGS = 30
if NBH_COL is not None:
    grp = eda.groupby(NBH_COL, observed=True)["price"].agg(mean="mean", n="size")
    top = grp[grp["n"] >= MIN_LISTINGS].sort_values("mean", ascending=False).head(15)
    plt.figure(figsize=(10,4))
    plt.bar(top.index, top["mean"])
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Average price (€)")
    plt.title(f"Top neighbourhoods by average price (min {MIN_LISTINGS} listings)")
    plt.tight_layout()
    plt.show()
else:
    print("No neighbourhood column found in this snapshot.")

## Price vs room type

Boxplots by `room_type` show a clear gradient:
- **Entire home/apt** → highest median and spread
- **Hotel room / Private room** → lower medians
- **Shared room** → lowest

This confirms `room_type` as a useful predictor.

In [ ]:
if "room_type" in eda.columns:
    order = [x for x in ["Entire home/apt","Hotel room","Private room","Shared room"]
             if x in eda["room_type"].unique()]
    eda = eda.copy()
    eda["room_type"] = pd.Categorical(eda["room_type"], categories=order, ordered=True)
    plt.figure(figsize=(8,4))
    eda.boxplot(column="price", by="room_type", grid=False)
    plt.ylim(0, 500)
    plt.title("Price by room_type")
    plt.suptitle("")
    plt.xlabel("")
    plt.ylabel("Price (€)")
    plt.tight_layout()
    plt.show()

## Export features for modeling

I export a compact table with:  
`room_type`, *neighbourhood*, `accommodates`, `bedrooms`, `bathrooms_num`,  
`minimum_nights`, `number_of_reviews`, `reviews_per_month`, `availability_365`, and target `price`.

Saved to: `data/berlin_clean.csv`.

In [ ]:
candidates = [
    "room_type", NBH_COL, "accommodates", "bedrooms", "bathrooms_num",
    "minimum_nights", "number_of_reviews", "reviews_per_month", "availability_365"
]
features = [c for c in candidates if c and c in df.columns]
target = "price"
data_clean = df[features + [target]].dropna()
assert len(data_clean) > 0, "No rows to export after dropna(). Check cleaning/feature list."

out_path = "../data/berlin_clean.csv"
Path(out_path).parent.mkdir(parents=True, exist_ok=True)
data_clean.to_csv(out_path, index=False)
print("Saved:", out_path, "Shape:", data_clean.shape)